In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import Metric

## Conclusion
Result of Midas model is lower than trained tf model because some groundtruth depthmap has reverse near-far value

---

In [ ]:
e1_res = pd.read_csv('metric_tfmodel.csv')
e2_res = pd.read_csv('metric_midas.csv')
e1_res.shape, e2_res.shape

In [ ]:
e1_res.mean()

In [ ]:
e2_res.mean()

In [ ]:
res_compare = pd.merge(e1_res, e2_res, on='img_path', how='inner', suffixes=('_e1', '_e2'))

In [ ]:
check_metric = 'mean_absrel'
res_compare['diff_{}'.format(check_metric)] = res_compare['{}_e1'.format(check_metric)] - res_compare['{}_e2'.format(check_metric)]
res_compare

In [ ]:
res_compare['diff_{}'.format(check_metric)].max(), res_compare['diff_{}'.format(check_metric)].min()

### Load sample data

In [ ]:
res_compare.nsmallest(5, 'diff_mean_absrel')

In [ ]:
choice = 19
print(res_compare.iloc[choice])
sample = res_compare.iloc[choice].img_path
sample

### Load both models to check result

In [ ]:
import torch
from tensorflow import keras
from e1_data_generator import DataGenerator
from utils import tf_visualize_depth_map, process_depth_map

In [ ]:
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 1

##### Load tf model

In [ ]:
tf_model = keras.models.load_model('checkpoint/model_1653373548.903842/')

##### Load torch Midas model

In [ ]:
model_type = "DPT_Hybrid"

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

##### tf pred

In [ ]:
sample_df = pd.DataFrame({'image': [sample], 'depth':[sample[:-4] + '_depth.npy'], 'mask': [sample[:-4] + '_depth_mask.npy']})
sample_df

In [ ]:
test_loader = DataGenerator(
    data=sample_df.reset_index(drop="true"), batch_size=BATCH_SIZE, dim=(HEIGHT, WIDTH), shuffle=False, is_test=True
)

In [ ]:
img_path, inp, target = test_loader[0]
tf_pred = tf_model.predict(inp)
tf_pred = tf_pred[0]

##### torch midas pred

In [ ]:
def general_normalize(data, origin_range, target_range):
    ori_min, ori_max = origin_range
    tar_min, tar_max = target_range
    return tar_min + (data - ori_min) * ((tar_max - tar_min) / (ori_max - ori_min))

In [ ]:
with torch.no_grad():
    image_ = cv2.imread(sample)
    image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
    input_image = transform(image_).to(device)
    h, w = input_image.shape[-2:]

#     depth_map = process_depth_map(sample[:-4] + '_depth.npy', sample[:-4] + '_depth_mask.npy', (w,h))

    pred = midas(input_image)
    pred_np = pred.permute(1, 2, 0).numpy()
    norm_pred = general_normalize(pred_np, (pred_np.min(), pred_np.max()), (target.min(), target.max()))
norm_pred.shape

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(18, 18))
cmap = plt.cm.jet
cmap.set_bad(color="black")

ax[0].imshow(image_)
ax[0].title.set_text('image')
ax[1].imshow(target[0], cmap=cmap)
ax[1].title.set_text('groundtruth')
ax[2].imshow(tf_pred, cmap=cmap)
ax[2].title.set_text('tf pred')
ax[3].imshow(norm_pred, cmap=cmap)
ax[3].title.set_text('midas pred')

In [ ]:
norm_pred_reshaped = np.expand_dims(cv2.resize(norm_pred, (256, 256)), -1)
fixed_norm_pred = -norm_pred_reshaped + norm_pred_reshaped.min() + norm_pred_reshaped.max()
plt.imshow(fixed_norm_pred, cmap=cmap)

In [ ]:
metric = Metric(target[0], fixed_norm_pred)
print(metric.mean_absrel_err())
print(metric.mean_sqrel_err())
print(metric.rms_err())
print(metric.rms_log_err())
print(metric.percentage_of_pixel_acc()['delta1'])
print(metric.percentage_of_pixel_acc()['delta2'])
print(metric.percentage_of_pixel_acc()['delta3'])

In [ ]:
res_compare.iloc[choice][['mean_absrel_e2', 'mean_sqrel_e2', 'rms_e2', 'rms_log_e2', 'delta1_e2', 'delta2_e2', 'delta3_e2']]